In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 34.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595719 sha256=06eb50641df23b704e89d1501b24064e8dbe05aea7ee0cb0b7f9dcb9652052e4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 35.1 MB/s 
     |████████████████████████████████| 769 kB 37.0 MB/s 
     |████████████████████████████████| 895 kB 67.9 MB/s 
     |████████████████████████████████| 3.0 MB 64.4 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6f_94igu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/

In [2]:
# 띄어쓰기
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

# 맞춤법
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-wcfl5w2p
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-wcfl5w2p
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2255828 sha256=195b424657e69cb344a0f9bc3f7278c49c5ab32b2cf5c6ffddeda57d78f29762
  Stored in directory: /tmp/pip-ephem-wheel-cache-revui114/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
Successfully built pykospacing


  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-gsd3dev0
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-gsd3dev0
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=7ef348998742283fc5d52dd67e57b0965f15a38a2e3cdca88d9ad7f8339c84fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-44ox88_n/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import re
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
#GPU 사용
device = torch.device("cuda:0")
print(device)

cuda:0


In [5]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [6]:
data=pd.read_excel('/content/공모전_제공_데이터(1차).xlsx')
classification=pd.read_excel('/content/공모전_제공_데이터(1차).xlsx', sheet_name='우선순위')

In [7]:
# 구두점 제거
print("구두점 제거")
only_word = []
for i in tqdm(range(len(data))):
  tmp = re.compile('[^가-힣a-zA-Z0-9 | ㄱ-ㅎ | ㅏ-ㅣ]').sub("",data.loc[i,'발화'])
  only_word.append(tmp)

# 띄어쓰기 제거
print("띄어쓰기 제거")
no_space = []
for i in tqdm(range(len(data))):
  tmp = re.sub(r"\s+", "", only_word[i])
  no_space.append(tmp)

# 띄어쓰기
from pykospacing import Spacing
print("띄어쓰기")
spc_data = []
for i in tqdm(range(len(data))):
  spacing = Spacing()
  tmp = spacing(no_space[i]) 
  spc_data.append(tmp)

# 맞춤법 교정
from hanspell import spell_checker
print("맞춤법교정")
spell_data = []
for i in tqdm(range(len(data))):
  tmp = spell_checker.check(spc_data[i])
  hanspell_tmp = tmp.checked
  spell_data.append(hanspell_tmp)

100%|██████████| 1000/1000 [00:00<00:00, 200934.37it/s]

구두점 제거
띄어쓰기 제거



  0%|          | 0/1000 [00:00<?, ?it/s]

띄어쓰기


  0%|          | 0/1000 [00:00<?, ?it/s]

맞춤법교정


100%|██████████| 1000/1000 [03:17<00:00,  5.07it/s]


In [8]:
spell_data

['상담직원과 빠른 연결했음 좋겠어요',
 '안 기다리고 빨리 상담되었으면 좋겠다',
 '너무 오래 기다리고 상담이 늦어져요',
 '상담직원과 통화가 매우 힘들다',
 '상담받으려면 오래 기다리는 게 불편함',
 '상담하려면 많이 기다려야 하는 게 불편합니다',
 '상담원이 통화 중이라 많이 기다려야 해요',
 '상담직원과의 연결이 빠르게 연결되면 촣겠습니다',
 'ars 상담직원 너무 많이 기다려야 하는 불편해요',
 '상담원 통화가 너 믜 힘듦',
 '상담원과의 연결이 빨리 되었으면 감사 힘니다',
 '상담원이 영결이 많이 힘이 듦',
 '5번 질문에 대한 답변은 상담직원 연결되기까지 기다리는 것 말곤 없는 듯',
 '상담을 하려면 좀 많이 기다려야 상담사분이 나오셔서 그게 좀 불편하네요',
 'Talk 상담이 다시 생겼으면',
 '상담하려 하는 기다리는 시간 너무 길다',
 '상담직원과 빠른 속도로 연결이 되길 원합니다',
 '상담직원과 빠른 연결 부탁드립니다',
 '상담직원과 통화하기가 힘들다',
 '상담 한 번 받으려면 보통 5분 이상 대기해야 실질 상담은 2분 미만인데',
 '직원 상담 시간 너무 지연됨 짜증 유발',
 '상담 톡이 부족',
 '상담사와 연결 부분이 어려웠어요',
 '톡 상담 인원을 늘려 주세요 요즘 같은 때에는 전화상담보다 톡 상담이 더 신속하고 일 처리가 빠른 데 인원이 너무 적은 듯 매번 기다리다 일 처리 못하고 그냥 지나간 적이 한두 번이 아닙니다',
 'ARS 지동 응답 불필요한 안내 축소 상담 직원과 빠른 연결을 원합니 다 추석 연휴 잘 보내세요',
 '1상담원과의 대화까지 소요되는 시간이 길어서 힘들었음 2 삼성카드사의 직원과 면대 면하여 질문 답변이 되길 수 있으면 좋겠습니다',
 '문자 상담이 나 온라인 상담이 가능한 부분이 많아졌으면 좋겠습니다',
 '상담직원과 연결이 좀 신속했으면 좋겠네요',
 '상담번호를 남겼는데 너무 늦게 주심이 너무 불만입니다',
 '상담직원 대기시간이 짫았으면 한다',
 '고객 입장에선 통

In [9]:
# 우선순위로 분류 # 1부터 44까지의 수로 분류 # 0은 없음
for i in range(len(classification)):
  data.loc[(data['최종분류(우선순위 가장 높은것 선택)'] == classification.loc[i,'의도명(유형)']), '최종분류(우선순위 가장 높은것 선택)'] = i+1

In [10]:
data_list = []
for q, label in zip(spell_data, data['최종분류(우선순위 가장 높은것 선택)'])  :
    dataf = []
    dataf.append(q)
    dataf.append(str(label))

    data_list.append(dataf)

In [11]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))                             

750
250


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 1000
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
batch_size = 1
num_epochs = 10
learning_rate =  0.01

In [15]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=44,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [19]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


RuntimeError: ignored